## Imports

In [1]:
import numpy as np 
import torch 
from torch.utils.data import Dataset, DataLoader
import json
from datetime import datetime

## Global varibles

In [2]:
DATASET = 'small16'
BATCH_SIZE = 32
OLD_MODEL = 'logreg-20230512T122508Z.txt'
INPUT_SIZE = 5
OUTPUT_SIZE = 1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 0.001
EPOCHS = 10

## Load data

In [3]:
class Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X, 
        self.y = y
    
    def __getitem__(self, idx):
        X = self.X[0][idx].astype('float32') 
        y = self.y[idx].astype('float32')
        return X, y
    
    def __len__(self):
        return len(self.y)

# load data
X_train = np.load(f'./data/{DATASET}/X_train.npy')
y_train = np.load(f'./data/{DATASET}/y_train.npy')
X_test = np.load(f'./data/{DATASET}/X_test.npy')
y_test = np.load(f'./data/{DATASET}/y_test.npy')

# make datasets
train_set = Dataset(X_train, y_train)
test_set = Dataset(X_test, y_test)

# make dataloaders
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)


## Load model

In [4]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

def load_model_from_txt(model, file_path):
    with open(file_path, 'r') as f:
        state_dict_json = json.load(f)
    state_dict = {k: torch.tensor(v) for k, v in state_dict_json.items()}
    model.load_state_dict(state_dict)
    return model

# init model
model = LogisticRegression(INPUT_SIZE, OUTPUT_SIZE)

# uncoment to load old model
model = load_model_from_txt(model, f'./models/{OLD_MODEL}')

# move model to device
model.to(DEVICE)

LogisticRegression(
  (linear): Linear(in_features=5, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## Train model

In [5]:
def train(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = torch.nn.BCELoss()
    model.train()
    losses = []
    for _ in range(EPOCHS):
        for X_train, y_true in train_loader:
            X_train = X_train.to(DEVICE)
            y_true = y_true.to(DEVICE)
            optimizer.zero_grad()
            y_pred = torch.squeeze(model(X_train))
            loss = criterion(y_pred, y_true)
            loss.backward()
            optimizer.step()        
            losses.append(loss.item())
    loss = sum(losses)/len(losses) 
    return model, loss

def test(model, test_loader):
    criterion = torch.nn.BCELoss()
    model.eval()
    losses = []
    with torch.no_grad():
        for X_test, y_true in test_loader:
            X_test = X_test.to(DEVICE)
            y_true = y_true.to(DEVICE)
            y_pred = torch.squeeze(model(X_test))
            loss = criterion(y_pred, y_true)
            losses.append(loss.item())
    loss = sum(losses)/len(losses) 
    return loss

test_loss_old = test(model, test_loader)
model, train_loss = train(model, train_loader)
test_loss = test(model, test_loader)

print(f'Train loss: {train_loss}')
print(f'Test loss from old model: {test_loss_old}')
print(f'Test loss from new model: {test_loss}')

Train loss: 0.22833787808194755
Test loss from old model: 0.6311156749725342
Test loss from new model: 0.6318812370300293


## Save new model

In [6]:
def save_model_to_txt(model, file_path):
    state_dict = model.state_dict()
    state_dict_json = {k: v.tolist() for k, v in state_dict.items()}
    with open(file_path, 'w') as f:
        json.dump(state_dict_json, f)

new_model_name = 'logreg' + '-' + datetime.utcnow().strftime('%Y%m%dT%H%M%S') + 'Z' + '.txt'
save_model_to_txt(model, f'./models/{new_model_name}')